In [8]:
! pip install bs4
! pip install lxml


  Using cached lxml-4.5.2-cp37-cp37m-manylinux1_x86_64.whl (5.5 MB)


In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [10]:
"""
Import Dietary reference intake info from
urls below into pandas df and make MongoDB DB
Recommended Dietary Allowances and Adequate Intakes
Must stay in bounds of upper limit for vitamins and minerals

"""
from bs4 import BeautifulSoup
import pandas as pd
import requests
import urllib.request
import copy

url_elements = 'https://www.ncbi.nlm.nih.gov/books/NBK545442/table/appJ_tab3/?report=objectonly'
url_vitamins = 'https://www.ncbi.nlm.nih.gov/books/NBK56068/table/summarytables.t2/?report=objectonly'
url_macroNutrients = 'https://www.ncbi.nlm.nih.gov/books/NBK56068/table/summarytables.t4/?report=objectonly'
url_upper_vitamins = 'https://www.ncbi.nlm.nih.gov/books/NBK56068/table/summarytables.t7/?report=objectonly'
url_upper_elements = 'https://www.ncbi.nlm.nih.gov/books/NBK545442/table/appJ_tab9/?report=objectonly'
url_macroNutrients_DistRange = "https://www.ncbi.nlm.nih.gov/books/NBK56068/table/summarytables.t5/?report=objectonly"

url_arr = [url_elements, url_vitamins, url_macroNutrients, url_upper_elements, url_upper_vitamins, 
           url_macroNutrients_DistRange]

names_arr = ['elements', 'vitamins', 'macronutrients', 'elements upper', 'vitamins upper', 
             'macronutrients dist range']
df_dict = {}
df_arr = []
for url in url_arr:
    soup = BeautifulSoup(requests.get(url).content, 'html.parser')

    # remove <sup>
    for sup in soup.select('sup'):
        sup.extract()
    df = pd.read_html(str(soup))[0]
    df_arr.append(df)
    



ImportError: lxml not found, please install it

In [194]:
# make copies so I don't have to download again in case of error
df_arr_copy = copy.deepcopy(df_arr)

In [1]:
# start by copying above copy here
df_arr = copy.deepcopy(df_arr_copy)

NameError: name 'copy' is not defined

In [231]:
# zip the two arrs into dict

df_dict = dict(zip(names_arr, df_arr))
    
for k,v in df_dict.items():
    print(f"{k}\n {v}")

elements
    Life-Stage Group Calcium (mg/d) Chromium (μg/d) Copper (μg/d)  \
0           Infants        Infants         Infants       Infants   
1            0–6 mo           200*            0.2*          200*   
2           7–12 mo           260*            5.5*          220*   
3          Children       Children        Children      Children   
4             1–3 y            700             11*           340   
5             4–8 y           1000             15*           440   
6             Males          Males           Males         Males   
7            9–13 y           1300             25*           700   
8           14–18 y           1300             35*           890   
9           19–30 y           1000             35*           900   
10          31–50 y           1000             35*           900   
11          51–70 y           1000             30*           900   
12           > 70 y           1200             30*           900   
13          Females        Females    

In [232]:
# change the col headers on macronutrients dist range

mac_distrange_df = df_dict['macronutrients dist range']
mac_distrange_df_copy = mac_distrange_df.copy
mac_distrange_df.columns = ['macronutrients as % energy', 'child 1-3 y', 'child 4-18 y', 'adults']

print(mac_distrange_df)
#rint(mac_distrange_df_copy.iloc[:,0])


                          macronutrients as % energy child 1-3 y child 4-18 y  \
0                                                Fat       30–40        25–35   
1    n-6 polyunsaturated fatty acids (linoleic acid)        5–10         5–10   
2  n-3 polyunsaturated fatty acids (α-linolenic a...     0.6–1.2      0.6–1.2   
3                                       Carbohydrate       45–65        45–65   
4                                            Protein        5–20        10–30   

    adults  
0    20–35  
1     5–10  
2  0.6–1.2  
3    45–65  
4    10–35  


In [233]:
'''
names_arr = ['elements', 'vitamins', 'macronutrients', 'elements upper', 'vitamins upper', 
             'macronutrients dist range']
'''


"\nnames_arr = ['elements', 'vitamins', 'macronutrients', 'elements upper', 'vitamins upper', \n             'macronutrients dist range']\n"

In [234]:
for k,v in df_dict.items():
    print(f"{k}\n {v}")

elements
    Life-Stage Group Calcium (mg/d) Chromium (μg/d) Copper (μg/d)  \
0           Infants        Infants         Infants       Infants   
1            0–6 mo           200*            0.2*          200*   
2           7–12 mo           260*            5.5*          220*   
3          Children       Children        Children      Children   
4             1–3 y            700             11*           340   
5             4–8 y           1000             15*           440   
6             Males          Males           Males         Males   
7            9–13 y           1300             25*           700   
8           14–18 y           1300             35*           890   
9           19–30 y           1000             35*           900   
10          31–50 y           1000             35*           900   
11          51–70 y           1000             30*           900   
12           > 70 y           1200             30*           900   
13          Females        Females    

In [235]:
# extract numbers
new_df_arr = []
for _,df in df_dict.items():
    df = df.astype(str)
    df_copy = df.copy()
    for i in range(1, len(df.columns)):
        df_copy[df.columns[i]]=df_copy[df.columns[i]].str.extract('(\d+[.]?\d*)', expand=False) #replace(r'[^0-9]+','')
    new_df_arr.append(df_copy)
# check df's
for df in new_df_arr:
    print(df)

# zip the two arrs into dict
df_dict = dict(zip(names_arr, new_df_arr))

   Life-Stage Group Calcium (mg/d) Chromium (μg/d) Copper (μg/d)  \
0           Infants            NaN             NaN           NaN   
1            0–6 mo            200             0.2           200   
2           7–12 mo            260             5.5           220   
3          Children            NaN             NaN           NaN   
4             1–3 y            700              11           340   
5             4–8 y           1000              15           440   
6             Males            NaN             NaN           NaN   
7            9–13 y           1300              25           700   
8           14–18 y           1300              35           890   
9           19–30 y           1000              35           900   
10          31–50 y           1000              35           900   
11          51–70 y           1000              30           900   
12           > 70 y           1200              30           900   
13          Females            NaN             N

In [236]:
# replace the Life StageGroup Column with new column to get rid of em dash
# up to that age easier later
lifestage_dict = {'Life-Stage Group': ['Infants', '<= 0.5 y', '<= 1.0 y',
                                       'Children', '<= 3.0 y', '<= 8.0 y',
                                    'Males', '<= 13.0 y', '<= 18.0 y', '<= 30.0 y',
                                      '<= 50.0 y', '<= 70.0 y', '> 70.0 y',
                                       'Females', '<= 13.0 y', '<= 18.0 y', '<= 30.0 y',
                                      '<= 50.0 y', '<= 70.0 y', '> 70.0 y',
                                       'Pregnancy', '<= 18.0 y', '<= 30.0 y',
                                       '<= 50.0 y',
                                       'Lacation', '<= 18.0 y', '<= 30.0 y',
                                       '<= 50.0 y']}


new_col_df = pd.DataFrame(lifestage_dict) 
print(new_col_df)

   Life-Stage Group
0           Infants
1          <= 0.5 y
2          <= 1.0 y
3          Children
4          <= 3.0 y
5          <= 8.0 y
6             Males
7         <= 13.0 y
8         <= 18.0 y
9         <= 30.0 y
10        <= 50.0 y
11        <= 70.0 y
12         > 70.0 y
13          Females
14        <= 13.0 y
15        <= 18.0 y
16        <= 30.0 y
17        <= 50.0 y
18        <= 70.0 y
19         > 70.0 y
20        Pregnancy
21        <= 18.0 y
22        <= 30.0 y
23        <= 50.0 y
24         Lacation
25        <= 18.0 y
26        <= 30.0 y
27        <= 50.0 y


In [237]:
'''
names_arr = ['elements', 'vitamins', 'macronutrients', 'elements upper', 'vitamins upper', 
             'macronutrients dist range']
'''

# iteratively change all except last
for name,df in df_dict.items():
    if name!='macronutrients dist range':
        if name=='elements' or name=='elements upper':
            df['Life-Stage Group'] = new_col_df['Life-Stage Group']
        else:
            df['Life StageGroup'] = new_col_df['Life-Stage Group']
        

for k,v in df_dict.items():
    print(f"{k}\n {v}\n")

elements
    Life-Stage Group Calcium (mg/d) Chromium (μg/d) Copper (μg/d)  \
0           Infants            NaN             NaN           NaN   
1          <= 0.5 y            200             0.2           200   
2          <= 1.0 y            260             5.5           220   
3          Children            NaN             NaN           NaN   
4          <= 3.0 y            700              11           340   
5          <= 8.0 y           1000              15           440   
6             Males            NaN             NaN           NaN   
7         <= 13.0 y           1300              25           700   
8         <= 18.0 y           1300              35           890   
9         <= 30.0 y           1000              35           900   
10        <= 50.0 y           1000              35           900   
11        <= 70.0 y           1000              30           900   
12         > 70.0 y           1200              30           900   
13          Females            NaN    

In [238]:
#https://pythonforundergradengineers.com/unicode-characters-in-python.html
print('\u03B1')
print(df_dict['vitamins'].columns)
print(df_dict['vitamins upper'].columns)

α
Index(['Life StageGroup', 'Vitamin A(μg/d)', 'Vitamin C(mg/d)',
       'Vitamin D(μg/d)', 'Vitamin E(mg/d)', 'Vitamin K(μg/d)',
       'Thiamin(mg/d)', 'Riboflavin(mg/d)', 'Niacin(mg/d)', 'Vitamin B6(mg/d)',
       'Folate(μg/d)', 'Vitamin B12(μg/d)', 'Pantothenic Acid(mg/d)',
       'Biotin(μg/d)', 'Choline(mg/d)'],
      dtype='object')
Index(['Life StageGroup', 'Vitamin A(μg/d)', 'Vitamin C(mg/d)',
       'Vitamin D(μg/d)', 'Vitamin E(mg/d)', 'Vitamin K', 'Thiamin',
       'Riboflavin', 'Niacin(mg/d)', 'Vitamin B6(mg/d)', 'Folate(μg/d)',
       'Vitamin B12', 'Pantothenic Acid', 'Biotin', 'Choline(g/d)',
       'Carotenoids'],
      dtype='object')


In [239]:
for k,v in df_dict.items():
    print(f"{k}\n {v}\n")

elements
    Life-Stage Group Calcium (mg/d) Chromium (μg/d) Copper (μg/d)  \
0           Infants            NaN             NaN           NaN   
1          <= 0.5 y            200             0.2           200   
2          <= 1.0 y            260             5.5           220   
3          Children            NaN             NaN           NaN   
4          <= 3.0 y            700              11           340   
5          <= 8.0 y           1000              15           440   
6             Males            NaN             NaN           NaN   
7         <= 13.0 y           1300              25           700   
8         <= 18.0 y           1300              35           890   
9         <= 30.0 y           1000              35           900   
10        <= 50.0 y           1000              35           900   
11        <= 70.0 y           1000              30           900   
12         > 70.0 y           1200              30           900   
13          Females            NaN    

In [240]:
# seems buggy so change again 
# delete cartoenoid from vitamins upper
#vitamins_upper = df_dict['vitamins upper']
#del vitamins_upper['Carotenoids']

for name,df in df_dict.items():
    if name=='vitamins': 
        df['Life StageGroup'] = new_col_df['Life-Stage Group']
        # change column names
        df.columns = ['Life-Stage Group', 'Vitamin A (\u03BCg/d)', 
                      'Vitamin C (mg/d)', 'Vitamin D (\u03BCg/d)',
                     'Vitamin E (mg/d)', 'Vitamin K (\u03BCg/d)', 
                      'Thiamin (mg/d)', 'Riboflavin (mg/d)', 'Niacin (mg/d)',
                     'Vitamin B6 (mg/d)', 'Folate (\u03BCg/d)', 'Vitamin B12 (\u03BCg/d)',
                     'Pantothenic Acid (mg/d)', 'Biotin (\u03BCg/d)', 'Choline (mg/d)']
    elif name=='vitamins upper':
        # change column names
        df.columns = ['Life-Stage Group', 'Vitamin A (\u03BCg/d)', 
                      'Vitamin C (mg/d)', 'Vitamin D (\u03BCg/d)',
                     'Vitamin E (mg/d)', 'Vitamin K (\u03BCg/d)', 
                      'Thiamin (mg/d)', 'Riboflavin (mg/d)', 'Niacin (mg/d)',
                     'Vitamin B6 (mg/d)', 'Folate (\u03BCg/d)', 'Vitamin B12 (\u03BCg/d)',
                     'Pantothenic Acid (mg/d)', 'Biotin (\u03BCg/d)', 'Choline (mg/d)',
                     'Carotenoids']
    elif name=='macronutrients':
        df.columns = ['Life-Stage Group', 'Total Water (L/d)', 'Carbohydrates (g/d)',
                     'Total Fiber (g/d)', 'Fat (g/d)', 'Linoleic Acid (g/d)', 
                     '\u03B1-Linolenic Acid (g/d)', 'Protein (g/d)']
for k,v in df_dict.items():
    print(f"{k}\n {v}\n")


elements
    Life-Stage Group Calcium (mg/d) Chromium (μg/d) Copper (μg/d)  \
0           Infants            NaN             NaN           NaN   
1          <= 0.5 y            200             0.2           200   
2          <= 1.0 y            260             5.5           220   
3          Children            NaN             NaN           NaN   
4          <= 3.0 y            700              11           340   
5          <= 8.0 y           1000              15           440   
6             Males            NaN             NaN           NaN   
7         <= 13.0 y           1300              25           700   
8         <= 18.0 y           1300              35           890   
9         <= 30.0 y           1000              35           900   
10        <= 50.0 y           1000              35           900   
11        <= 70.0 y           1000              30           900   
12         > 70.0 y           1200              30           900   
13          Females            NaN    

In [241]:
'''
# delete extra column 
for name, df in df_dict.items():
        if name=='elements' or name=='elements upper':
            del df['Life StageGroup']


for k,v in df_dict.items():
    print(f"{k}\n {v}\n")
'''

'\n# delete extra column \nfor name, df in df_dict.items():\n        if name==\'elements\' or name==\'elements upper\':\n            del df[\'Life StageGroup\']\n\n\nfor k,v in df_dict.items():\n    print(f"{k}\n {v}\n")\n'

In [242]:
#append units to cells
for _,df in df_dict.items():
    for col in df.columns:
        if col != 'Life-Stage Group':
            if 'mg' in col:
                df[col] = df[col].astype(str) + ' mg'
            elif '\u03BC' in col:
                df[col] = df[col].astype(str) + ' \u03BCg'
            elif 'L/d' in col:
                df[col] = df[col].astype(str) + ' L'
            else: # 'g'
                df[col] = df[col].astype(str) + ' g'

for k,v in df_dict.items():
    print(f"{k}\n {v}\n")    

elements
    Life-Stage Group Calcium (mg/d) Chromium (μg/d) Copper (μg/d)  \
0           Infants         nan mg          nan μg        nan μg   
1          <= 0.5 y         200 mg          0.2 μg        200 μg   
2          <= 1.0 y         260 mg          5.5 μg        220 μg   
3          Children         nan mg          nan μg        nan μg   
4          <= 3.0 y         700 mg           11 μg        340 μg   
5          <= 8.0 y        1000 mg           15 μg        440 μg   
6             Males         nan mg          nan μg        nan μg   
7         <= 13.0 y        1300 mg           25 μg        700 μg   
8         <= 18.0 y        1300 mg           35 μg        890 μg   
9         <= 30.0 y        1000 mg           35 μg        900 μg   
10        <= 50.0 y        1000 mg           35 μg        900 μg   
11        <= 70.0 y        1000 mg           30 μg        900 μg   
12         > 70.0 y        1200 mg           30 μg        900 μg   
13          Females         nan mg    

In [228]:
print('epsilon: \u03B5')
print('mu: \u03BC')



epsilon: ε
mu: μ


In [243]:
my_df_dict = copy.deepcopy(df_dict)
#path = 'csvThanksgiving/'
# concatenate
'''
names_arr = ['elements', 'vitamins', 'macronutrients', 'elements upper', 'vitamins upper', 
             'macronutrients dist range']
'''
elements_df = my_df_dict['elements']
vitamins_df = my_df_dict['vitamins']
macronutrients_df = my_df_dict['macronutrients']
upper_elements_df = my_df_dict['elements upper']
upper_vitamins_df = my_df_dict['vitamins upper']
#macronutrients_distrange_df = my_df_dict['macronutrients dist range']

elements_df.to_csv(path+"elements_rdi.csv", index=False)
vitamins_df.to_csv(path+"vitamins_rdi.csv", index=False)
macronutrients_df.to_csv(path+"macronutrients_rdi.csv", index=False)
upper_elements_df.to_csv(path+"elements_upper_rdi.csv", index=False)
upper_vitamins_df.to_csv(path+"vitamins_upper_rdi.csv", index=False)
#macronutrients_distrange_df.to_csv(path+"macronutrients_dist_range.csv", index=False)




In [12]:
import pandas as pd
#read csv
elements_df = pd.read_csv('elements_rdi.csv')
vitamins_df = pd.read_csv('vitamins_rdi.csv')
macronutrients_df = pd.read_csv('macronutrients_rdi.csv')
upper_elements_df = pd.read_csv('elements_upper_rdi.csv')
upper_vitamins_df = pd.read_csv('vitamins_upper_rdi.csv')

In [18]:
print(f'{elements_df}\n{vitamins_df}\n,{macronutrients_df}\n,{upper_elements_df}\n,{upper_vitamins_df}')

   Life-Stage Group Calcium (mg/d) Chromium (μg/d) Copper (μg/d)  \
0           Infants         nan mg          nan μg        nan μg   
1          <= 0.5 y         200 mg          0.2 μg        200 μg   
2          <= 1.0 y         260 mg          5.5 μg        220 μg   
3          Children         nan mg          nan μg        nan μg   
4          <= 3.0 y         700 mg           11 μg        340 μg   
5          <= 8.0 y        1000 mg           15 μg        440 μg   
6             Males         nan mg          nan μg        nan μg   
7         <= 13.0 y        1300 mg           25 μg        700 μg   
8         <= 18.0 y        1300 mg           35 μg        890 μg   
9         <= 30.0 y        1000 mg           35 μg        900 μg   
10        <= 50.0 y        1000 mg           35 μg        900 μg   
11        <= 70.0 y        1000 mg           30 μg        900 μg   
12         > 70.0 y        1200 mg           30 μg        900 μg   
13          Females         nan mg          nan 

In [21]:
#elements divided by rows, skip header rows
infants_elements = elements_df.iloc[1:3, :]
children_elements = elements_df.iloc[4:6, :]
males_elements = elements_df.iloc[7:13, :]
females_elements = elements_df.iloc[14:20, :]
pregnancy_elements = elements_df.iloc[21:24, :]
lactation_elements = elements_df.iloc[25:, :]
# same rows as elements, don't skip lifestage col
infants_vitamins = vitamins_df.iloc[1:3, :]
children_vitamins = vitamins_df.iloc[4:6, :]
males_vitamins = vitamins_df.iloc[7:13, :]
females_vitamins = vitamins_df.iloc[14:20, :]
pregnancy_vitamins = vitamins_df.iloc[21:24, :]
lactation_vitamins = vitamins_df.iloc[25:, :]
#macro same, 28 * 8, skip lifestage col
infants_macronutrients = macronutrients_df.iloc[1:3, :]
children_macronutrients = macronutrients_df.iloc[4:6, :]
males_macronutrients = macronutrients_df.iloc[7:13, :]
females_macronutrients = macronutrients_df.iloc[14:20, :]
pregnancy_macronutrients = macronutrients_df.iloc[21:24, :]
lactation_macronutrients = macronutrients_df.iloc[25:, :]

# upper_minerals, 28 * 22
infants_upper_elements = upper_elements_df.iloc[1:3, :]
children_upper_elements = upper_elements_df.iloc[4:6, :]
males_upper_elements = upper_elements_df.iloc[7:13, :]
females_upper_elements = upper_elements_df.iloc[14:20, :]
pregnancy_upper_elements = upper_elements_df.iloc[21:24, :]
lactation_upper_elements = upper_elements_df.iloc[25:, :]

#upper_vitamins same 28 *16, skip lifestage col
infants_upper_vitamins = upper_vitamins_df.iloc[1:3, 1:]
children_upper_vitamins = upper_vitamins_df.iloc[4:6, 1:]
males_upper_vitamins = upper_vitamins_df.iloc[7:13, 1:]
females_upper_vitamins = upper_vitamins_df.iloc[14:20, 1:]
pregnancy_upper_vitamins = upper_vitamins_df.iloc[21:24, 1:]
lactation_upper_vitamins = upper_vitamins_df.iloc[25:, 1:]

In [245]:
'''
#concat on axis 1
infants_micro_rdi = pd.concat([infant_elements, infant_vitamins], axis=1, sort=False)
children_rdi = pd.concat([child_elements, child_vitamins], axis=1, sort=False)
males_rdi = pd.concat([male_elements, male_vitamins], axis=1, sort=False)
females_rdi = pd.concat([female_elements, female_vitamins], axis=1, sort=False)
pregnancy_rdi = pd.concat([preg_elements, preg_vitamins], axis=1, sort=False)
lactation_rdi = pd.concat([lact_elements, lact_vitamins], axis=1, sort=False)


# concat upper ranges
infants_upper_micro = pd.concat([infant_upper_elements, infant_upper_vitamins], axis=1, sort=False)
children_upper_micro = pd.concat([child_upper_elements, child_upper_vitamins], axis=1, sort=False)
males_upper_micro = pd.concat([male_upper_elements, male_upper_vitamins], axis=1, sort=False)
females_upper_micro = pd.concat([female_upper_elements, female_upper_vitamins], axis=1, sort=False)
preg_upper_micro = pd.concat([preg_upper_elements, preg_upper_vitamins], axis=1, sort=False)
lact_upper_micro = pd.concat([lact_upper_elements, lact_upper_vitamins], axis=1, sort=False)

# yank out u4, u18 and adults
#macronutrients_distrange_df.columns = ['macronutrients as % energy', 'child 1-3 y', 'child 4-18 y', 'adults']
u4_macro_range = macronutrients_distrange_df[['macronutrients as % energy', 'child 1-3 y']].copy()
u18_macro_range = macronutrients_distrange_df[['macronutrients as % energy', 'child 4-18 y']].copy()
adults_macro_range = macronutrients_distrange_df[['macronutrients as % energy', 'adults']].copy()
'''

In [22]:
infants_elements.to_csv("infants_elements.csv", index=False)
infants_vitamins.to_csv("infants_vitamins.csv", index=False)
infants_macronutrients.to_csv("infants_macronutrients.csv", index=False)
infants_upper_elements.to_csv("infants_upper_elements.csv", index=False)
infants_upper_vitamins.to_csv("infants_upper_vitamins.csv", index=False)

children_elements.to_csv("children_elements.csv", index=False)
children_vitamins.to_csv("children_vitamins.csv", index=False)
children_macronutrients.to_csv("children_macronutrients.csv", index=False)
children_upper_elements.to_csv("children_upper_elements.csv", index=False)
children_upper_vitamins.to_csv("children_upper_vitamins.csv", index=False)

males_elements.to_csv("males_elements.csv", index=False)
males_vitamins.to_csv("males_vitamins.csv", index=False)
males_macronutrients.to_csv("males_macronutrients.csv", index=False)
males_upper_elements.to_csv("males_upper_elements.csv", index=False)
males_upper_vitamins.to_csv("males_upper_vitamins.csv", index=False)

females_elements.to_csv("females_elements.csv", index=False)
females_vitamins.to_csv("females_vitamins.csv", index=False)
females_macronutrients.to_csv("females_macronutrients.csv", index=False)
females_upper_elements.to_csv("females_upper_elements.csv", index=False)
females_upper_vitamins.to_csv("females_upper_vitmains.csv", index=False)

pregnancy_elements.to_csv("pregnancy_elements.csv", index=False)
pregnancy_vitamins.to_csv("pregnancy_vitamins.csv", index=False)
pregnancy_macronutrients.to_csv("pregnancy_macronutrients.csv", index=False)
pregnancy_upper_elements.to_csv("pregnancy_upper_elements.csv", index=False)
pregnancy_upper_vitamins.to_csv("pregnancy_upper_vitamins.csv", index=False)

lactation_elements.to_csv("lactation_elements.csv", index=False)
lactation_vitamins.to_csv("lactation_vitamins.csv", index=False)
lactation_macronutrients.to_csv("lactation_macronutrients.csv", index=False)
lactation_upper_elements.to_csv("lactation_upper_elements.csv", index=False)
lactation_upper_vitamins.to_csv("lactation_upper_vitamins.csv", index=False)

'''
# concatenated upper elem, vit
infants_upper_micro.to_csv(path+"infants_upper_micro.csv", index=False)
children_upper_micro.to_csv(path+"children_upper_micro.csv", index=False)
males_upper_micro.to_csv(path+"males_upper_micro.csv", index=False)
females_upper_micro.to_csv(path+"females_upper_micro.csv", index=False)
preg_upper_micro.to_csv(path+"preg_upper_micro.csv", index=False)
lact_upper_micro.to_csv(path+"lact_upper_micro.csv", index=False)

#u4_macro_range.to_csv(path+"u4_macro_range.csv", index=False)
#u18_macro_range.to_csv(path+"u18_macro_range.csv", index=False)
#adults_macro_range.to_csv(path+"adults_macro_range.csv", index=False)
'''




'\n# concatenated upper elem, vit\ninfants_upper_micro.to_csv(path+"infants_upper_micro.csv", index=False)\nchildren_upper_micro.to_csv(path+"children_upper_micro.csv", index=False)\nmales_upper_micro.to_csv(path+"males_upper_micro.csv", index=False)\nfemales_upper_micro.to_csv(path+"females_upper_micro.csv", index=False)\npreg_upper_micro.to_csv(path+"preg_upper_micro.csv", index=False)\nlact_upper_micro.to_csv(path+"lact_upper_micro.csv", index=False)\n\n#u4_macro_range.to_csv(path+"u4_macro_range.csv", index=False)\n#u18_macro_range.to_csv(path+"u18_macro_range.csv", index=False)\n#adults_macro_range.to_csv(path+"adults_macro_range.csv", index=False)\n'

In [2]:
'''
macronutrients dist range
                           macronutrients as % energy child 1-3 y child 4-18 y  \
0                                              Fat g        30 g         25 g   
1  n-6 polyunsaturated fatty acids (linoleic acid) g         5 g          5 g   
2  n-3 polyunsaturated fatty acids (α-linolenic a...       0.6 g        0.6 g   
3                                     Carbohydrate g        45 g         45 g   
4                                          Protein g         5 g         10 g   

  adults  
0   20 g  
1    5 g  
2  0.6 g  
3   45 g  
4   10 g  


'''
import pandas as pd
cols_arr = ['Life-Stage Group','Fat', 'Linoleic Acid', '\u03B1-Linolenic Acid', 'Carbohydrate', 'Protein']
macro_distrange_dict = {
    'Life-Stage Group': ['child <= 3 y', 'child <= 18 y', 'adults'],
    'Fat':['30 %', '25 %', '20 %'],
    'Linoleic Acid':['5 %', '5 %', '5 %'],
    '\u03B1-Linolenic Acid': ['0.6 %', '0.6 %', '0.6 %'],
    'Carbohydrate': ['45 %', '45 %', '45 %'],
    'Protein': ['5 %', '10 %', '10 %']
}

macro_distrange_df = pd.DataFrame(macro_distrange_dict)

print(macro_distrange_df)

  Life-Stage Group   Fat Linoleic Acid α-Linolenic Acid Carbohydrate Protein
0     child <= 3 y  30 %           5 %            0.6 %         45 %     5 %
1    child <= 18 y  25 %           5 %            0.6 %         45 %    10 %
2           adults  20 %           5 %            0.6 %         45 %    10 %


In [3]:
# write to csv
macro_distrange_df.to_csv(path"all_macro_range.csv", index=False)

NameError: name 'path' is not defined